# EDA + find groups of variants in training dataset

# Introduction

The main goal of this notebook is to check if there is variants of a same protein in the training dataset and to regroup them. 
An other goal is to create new features. I'll use a corrected version of the train.csv file.

# Import packages

In [ ]:
# data processing
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# compute distances
from scipy.spatial.distance import hamming
from sklearn.metrics import pairwise_distances
from Levenshtein import distance as lev

# time
import timeit

In [ ]:
pd.set_option("display.max_columns", 30)

# Load data

In [ ]:
train = pd.read_csv("Data/train_correct.csv") 
test = pd.read_csv("Data/test.csv")

In [ ]:
print("train shape : ", train.shape)
print("test shape : ", test.shape)

In [ ]:
train.head()

In [ ]:
test.head()

Proteins of the test file are variants of a same enzyme so proteins sequences are very similar. It is not the case of the training dataset.

# Cleaning

We don't need 'data_source', and there is duplicates and Nan.

In [ ]:
train.drop('data_source', axis=1, inplace=True)
test.drop('data_source', axis=1, inplace=True)

In [ ]:
train.drop_duplicates(subset='protein_sequence', inplace=True)
train.dropna(inplace=True)
train.shape

# pH and tm

In [ ]:
train[['pH', 'tm']].describe()

In [ ]:
sns.histplot(data=train, x='pH')

In [ ]:
sns.histplot(data=train, x='tm')

In [ ]:
test['pH'].value_counts()

# Protein sequence

There is two different ways to get more features.

First we can download pdb files for proteins in train and extract informations from those files. But my knowledge in biology is very limited and i don't know what informations are relevant.

So I'll use another approach: considering protein sequence simply as a sequence of letters, without biological context.

## Length of sequences

In [ ]:
train['protein_sequence'].iloc[0]

In [ ]:
train['protein_length'] = train['protein_sequence'].apply(lambda x: len(x))

In [ ]:
sns.histplot(data=train, x='protein_length')

Proteins in test must have approximately the same length.

In [ ]:
test['protein_sequence'].apply(lambda x: len(x)).value_counts()

I'll consider only protein of length less than 1000 in train because proteins of very different length can't be variants of a same protein.

In [ ]:
train = train[train['protein_length']<1000]
train.shape

In [ ]:
sns.histplot(data=train, x='protein_length')

## Amino acid

In [ ]:
amino_count = train['protein_sequence'].str.split('').explode('protein_sequence').value_counts().drop('')
amino_count

In [ ]:
len(amino_count.index)

In [ ]:
sns.barplot(x=amino_count.index, y=amino_count)

In [ ]:
# for later
amino_list = list(amino_count.index)
amino_list.sort()

## Variants

The correct distance to find variants is Levenshtein's distance but it's not implemented with 'paiwise_distances' of scikit-learn. Hamming's distance is not correct and we will see why.

First we will test those distances on test set because we know that all proteins are variants of the same enzyme.

### test

In [ ]:
test

### Hamming's distance

It doesn't work with strings.

In [ ]:
df = pd.DataFrame(test['protein_sequence'].apply(list).tolist())
df

In [ ]:
amino_dict = {k: i+1 for i, k in enumerate(amino_list)}
amino_dict[None] = 0

In [ ]:
df = df.replace(amino_dict)
df

In [ ]:
X = df.values

In [ ]:
start_time = timeit.default_timer()

hamming_test_matrix = pairwise_distances(X, metric=hamming)
    
temps = timeit.default_timer() - start_time
print(temps)

In [ ]:
hamming_test_matrix

There is hight distances due to a shift in sequences. We can see it at the end of the third row (index 2).

In [ ]:
df.head()

### Levenshtein's distance

In [ ]:
n = test.shape[0]
levenshtein_test_matrix = np.full((n, n), 0)

In [ ]:
start_time = timeit.default_timer()
for i in range(n):
    for j in range(i+1, n):
        levenshtein_test_matrix[i][j] = lev(test['protein_sequence'].iloc[i], test['protein_sequence'].iloc[j])
temps = timeit.default_timer() - start_time
print(temps)

In [ ]:
# estimation of time to compute for train, in minutes
temps*100/60

In [ ]:
levenshtein_test_matrix = levenshtein_test_matrix + levenshtein_test_matrix.T
levenshtein_test_matrix

In [ ]:
levenshtein_test_matrix.max()

So we can consider further (in train) that two proteins are variants if the Levenshtein's distance between them is less or equal to 2, maybe 3.

### train

We can reasonably think that the first 20 characters are sufficient to check if proteins are variants because there is much more combinations (with repetitions) of 20 characters in a set of 20 characters than there is proteins in train:

In [ ]:
20**20

In [ ]:
def tronc_str(x, n):
    if len(x) < n:
        return x
    else:
        return x[:n]

In [ ]:
train['first_20'] = train['protein_sequence'].apply(lambda x: tronc_str(x, 20))

In [ ]:
train.head()

We only want to regroup variants, not to get all distances. In the cell below we create a dictionary wich regroup sequence's id of variants (of a same protein) at the same key (in a list).

In [ ]:
copy_train = train.copy()
dic = {}
n_group = 1

start_time = timeit.default_timer()

while copy_train.shape[0] > 0 :
    l = []
    a = copy_train['first_20'].iloc[0]
    for i in copy_train.index:
        if lev(a, copy_train['first_20'].loc[i])<= 3:
            l.append(i)
    dic[n_group] = l
    n_group += 1
    copy_train.drop(l, axis=0, inplace=True)
    
temps = timeit.default_timer() - start_time
print(temps / 60)


In [ ]:
liste = [dic[i] for i in dic.keys()]
d = {'col1': liste}
groups = pd.DataFrame(data=d, index=dic.keys())
groups

And we add a new feature in train.

In [ ]:
train['group'] = np.nan

In [ ]:
for i in range(groups.shape[0]):
    for j in groups['col1'].values[i]:
        train['group'].loc[j] = i

In [ ]:
train

Let us check if there is in train a variant of the enzyme in test.

In [ ]:
l = []
for i in range(train.shape[0]):
    l.append(lev(test['protein_sequence'].iloc[0], train['protein_sequence'].iloc[i]))
min(l)

No, there is not.

In [ ]:
test['group'] = train['group'].max() + 1

## Groups of variants in train.

In [ ]:
group_train = train.groupby(['group'])
group_train = group_train.agg({
    'seq_id': 'count',
    'pH': 'mean',
    'tm': 'mean'
})

In [ ]:
group_train['seq_id'].value_counts()

There is only 4 groups with more than 100 variants. We will study them.

In [ ]:
group_train = group_train[group_train['seq_id'] > 100]
group_train

In [ ]:
# for example
train[train['group'] == 12462]

### pH and tm per group

In [ ]:
list_100 = list(group_train.index)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))

for i in range(2):
    sns.histplot(data=train[train['group'] == list_100[i]], x='pH', ax=axs[i, 0]).set(xlabel='pH: group ' + str(list_100[i]))

for i in range(2): 
    sns.histplot(data=train[train['group'] == list_100[i+2]], x='pH', ax=axs[i, 1]).set(xlabel='pH :group ' + str(list_100[i+2]))

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))

for i in range(2):
    sns.histplot(data=train[train['group'] == list_100[i]], x='tm', ax=axs[i, 0]).set(xlabel='tm: group ' + str(list_100[i]))

for i in range(2):
    sns.histplot(data=train[train['group'] == list_100[i+2]], x='tm', ax=axs[i, 1]).set(xlabel='tm :group ' + str(list_100[i+2]))

In [ ]:
meanprops = {'marker':'o', 'markeredgecolor':'black', 'markerfacecolor':'firebrick'}

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

sns.boxplot(x="pH", y="group", showfliers=False, showmeans=True, meanprops=meanprops,
            orient='h', data=train[train['group'].isin(list_100)], ax=axs[0])
sns.boxplot(x="tm", y="group", showfliers=False, showmeans=True, meanprops=meanprops,
            orient='h', data=train[train['group'].isin(list_100)], ax=axs[1])

So pH and tm can be very different for similar proteins.

## More features

### First option: Amino acid without order

In [ ]:
train1 = train.copy()

train1.drop(['first_20'], axis=1, inplace=True)

In [ ]:
def count_amino(prot, am):
    count = len([l for l in prot if l==am])
    return count

In [ ]:
for amino in amino_list:
    train1[amino] = train1['protein_sequence'].apply(lambda x: count_amino(x, amino))

In [ ]:
train1

### Second option: position as a feature

In [ ]:
train2 = pd.DataFrame(train['protein_sequence'].apply(list).tolist())
train2 = train2.replace(amino_dict)
train2 = pd.concat([train2, train.reset_index()['pH']], axis=1)
train2